# NIHCXR Detectron Experiments

## Import Libraries and Load NIHCXR Dataset

In [1]:
import numpy as np
from monai.transforms import (
    Compose,
    Lambdad,
    Resized,
)
from torch import nn
from torchxrayvision.models import DenseNet

from cyclops.data.loader import load_nihcxr
from cyclops.monitor import ClinicalShiftApplicator, DCTester

DATA_DIR = "/mnt/data/clinical_datasets/NIHCXR"

nih_ds = load_nihcxr(DATA_DIR)

## Generate Source/Target Dataset for Experiments

In [2]:
shifter = ClinicalShiftApplicator(
"sex", source=None, target="F", shift_id="Patient Gender",
)
_, target_ds = shifter.apply_shift(nih_ds["test"], num_proc=6)

source_ds = nih_ds
target_ds = target_ds.train_test_split(test_size=0.10)

source_ds["train"] = source_ds["train"].select(range(1000))
source_ds["test"] = source_ds["test"].select(range(1000))

target_ds["train"] = target_ds["train"].select(range(1000))
target_ds["test"] = target_ds["test"].select(range(1000))

transforms = Compose(
    [
        Resized(
            keys=("image",), spatial_size=(224, 224), allow_missing_keys=True,
        ),
        Lambdad(
            keys=("image",),
            func=lambda x: ((2 * (x / 255.0)) - 1.0) * 1024,
            allow_missing_keys=True,
        ),
        Lambdad(("image",), func=lambda x: np.mean(x, axis=0)[np.newaxis, :] if x.shape[0] != 1 else x),
    ],
)

Filter (num_proc=6):   0%|          | 0/25596 [00:00<?, ? examples/s]

## Initialize Base Model and Run experiment with Detectron


In [3]:
base_model = DenseNet(weights="densenet121-res224-nih")
base_model.classifier.weight = nn.Parameter(base_model.classifier.weight[:14])
base_model.classifier.bias = nn.Parameter(base_model.classifier.bias[:14])
base_model.op_threshs = base_model.op_threshs[:14]

detectron = DCTester("detectron",
                  base_model=base_model,
                  feature_column="image",
                  transforms=transforms,
                  task="multilabel",
                  sample_size=50,
                  max_epochs_per_model=5,
                  ensemble_size=5,
                  lr=0.01,
                  num_runs=20)

detectron.fit(source_ds)

p_val, distance = detectron.test_shift(target_ds)

Flattening the indices:   0%|          | 0/10 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/10 [00:00<?, ? examples/s]

2023-09-10 21:42:04,041 INFO cyclops.models.wrappers.pt_model - [1/1]                 Training loss: 0.1361 	
2023-09-10 21:42:04,692 INFO cyclops.models.wrappers.pt_model - [1/1]                     Validation loss: 0.1479 	
2023-09-10 21:42:04,692 INFO cyclops.models.wrappers.pt_model - Best model saved at epoch 1 in detectron/saved_models/DetectronModule


Flattening the indices:   0%|          | 0/10 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/10 [00:00<?, ? examples/s]

2023-09-10 21:42:07,099 INFO cyclops.models.wrappers.pt_model - [1/1]                 Training loss: 0.1484 	
2023-09-10 21:42:07,750 INFO cyclops.models.wrappers.pt_model - [1/1]                     Validation loss: 0.1489 	
2023-09-10 21:42:07,751 INFO cyclops.models.wrappers.pt_model - Best model saved at epoch 1 in detectron/saved_models/DetectronModule
